In [ ]:
import matplotlib.pyplot as plt
from pandas.plotting import parallel_coordinates
import seaborn as sns
import time

%matplotlib inline

In [ ]:
!pip install pyspark


In [ ]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext

from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col

In [ ]:
spark = SparkSession.builder.master("local[2]").appName("Linear-Regression-California-Housing").getOrCreate()

In [ ]:
data = spark.read.csv("../input/minute-weather" ,header=True, inferSchema=True)
data.printSchema()

In [ ]:
data.toPandas().shape

In [ ]:
from pyspark.sql.functions import isnan, when, count, col
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show()

In [ ]:
df =data[data['rowID']<10000]

In [ ]:
df.count()

In [ ]:
df = df.drop('rowID')
df = df.drop('hpwren_timestamp')
df = df.drop('air_pressure')
df = df.drop('avg_wind_speed')
df = df.drop('max_wind_speed')
df = df.drop('min_wind_speed')
df = df.drop('rain_accumulation')
df = df.drop('rain_duration')
df.show()

In [ ]:
from pyspark.sql.functions import isnan, when, count, col
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

In [ ]:
df.toPandas().describe()

In [ ]:
import pyspark.sql.functions as F


In [ ]:
# (df.describe().select(
#                     "summary",
#                     F.round("air_temp", 4).alias("air_temp"),
#                     F.round("avg_wind_direction", 4).alias("avg_wind_direction"),
#                     F.round("max_wind_direction", 4).alias("max_wind_direction"),
#                     F.round("min_wind_direction", 4).alias("min_wind_direction"),
#                     F.round("relative_humidity", 4).alias("relative_humidity"))
#                     .show())

In [ ]:
sns.pairplot(df.toPandas())

In [ ]:
df=df.na.drop()

In [ ]:
df.count()

In [ ]:
sns.histplot(data=df.toPandas(), x="air_temp")


In [ ]:
sns.histplot(data=df.toPandas(), x="avg_wind_direction")


In [ ]:
sns.histplot(data=df.toPandas(), x="min_wind_direction")


In [ ]:
sns.histplot(data=df.toPandas(), x="max_wind_direction")


In [ ]:
sns.histplot(data=df.toPandas(), x="relative_humidity")


In [ ]:
from pyspark.sql.functions import col , column
df = df.withColumn("air_temp", col("air_temp").cast("double"))
df = df.withColumn("avg_wind_direction", col("avg_wind_direction").cast("double"))
df = df.withColumn("avg_wind_direction", col("avg_wind_direction").cast("double"))
df = df.withColumn("min_wind_direction", col("min_wind_direction").cast("double"))
df = df.withColumn("relative_humidity", col("relative_humidity").cast("double"))

In [ ]:
from pyspark.ml.clustering import KMeans


In [ ]:
# import numpy as np
# cost = np.zeros(20)
# for k in range(2,20):
#     kmeans = KMeans()\
#             .setK(k)\
#             .setSeed(1) \
#             .setFeaturesCol(data[data['air_temp'])\
#             .setPredictionCol("cluster")

#     model_k = kmeans.fit(df)
#     cost[k] = model_k.computeCost(df)

In [ ]:
# random vi tri trung tam ban dau
def random_init(max_min,k,len_columns):
    import random
    # mang de luu vi tri trung tam cụm
    k_position=[]
    import random

    # lặp vị trí trung tâm trong từng cụm
    for i in range(0,k):
        row=[]
        # lưu lại vị trí trung tâm của cụm thứ i 
        for j in range(0,len_columns):
            row.append( random.randrange(int(max_min[1][j]),int(max_min[0][j])))
        k_position.append(row)
    return k_position

# tìm cụm gần nhất cho 1 điểm    
def find_closeness(position,k_position):
    min_closness = 0
    # cụm thứ index
    index = 0
    # len(k_position) là số k, k_position = [[1,3],[2,4]]; cụm [1,3] và cụm [2,4]
    for i in range (0 , len(k_position)):
      # i = 0 là cụm [1,3]
        distance = 0
        # tính khoảng cách từ điểm position đến các k_position
        for j in range (0,len(position)):
            distance = distance + (position[j]-k_position[i][j])**2
        if i == 0:
            min_closness = distance
        else:
            if distance < min_closness:
                min_closness = distance
                index = i
    return index

# update các vị trí trung tâm mới của cụm
def find_positions(df,k,name_columns):
  # tính giá trị trung bình của từng cụm
    # dữ liệu trả về có k dòng, 
    data = df.groupBy('group').avg().collect()
    #  k_position = [[1,3],[2,4]]; cụm [1,3] và cụm [2,4]
    k_position =[]
    lenght_columns = len(name_columns)

    # lặp theo từng cụm để lấy
    for i in range (0,k):
      # lưu lại giá trị từng dòng, dòng thứ 1 tương ứng vs giá trị trung bình của tất cả phần tử trong group=0
        index = [] 
# lặp theo cột để lấy giá trị ra
        for j in range (0,lenght_columns):
            name = 'avg(' + str(name_columns[j]) +')'
            index.append(data[i][name])
      #  k_position = [[1,3],[2,4]]; index = [1,3] và index = [2,4]
        k_position.append(index)
    return k_position
    
def k_mean(df,k):
    from pyspark.sql.functions import lit
    from pyspark.sql import Row
    import pyspark.sql.functions as F     
# đặt ở trong vì các biến này chỉ dùng trong hàm, vì nếu chạy lại à k khai báo thì k chạy dc
    def classtify(row):
        rd=row.asDict() 
        len_columns=len(k_position[0])
        # position là 1 điểm
        position=[]
        for i in range (0,len_columns):
            position.append(row[i])
# index là cụm; index = 0 là cụm 0
        index = find_closeness(position,k_position)
        if row['group'] == index:
            rd["status"]=0
        else:
            rd["status"]=1
        rd["group"]=index
        # add cột status vào df
        new_row=Row(**rd)  
        return new_row


    columns = df.columns
    len_columns = len(columns)
    max_value=[]
    min_value=[]
    max_min = []
    # lặp từng cột để lấy giá trị max min của từng cột
    for i in  columns:
        row_max = df.agg({i: "max"}).collect()[0]
        name_column = 'max('+i+')'
        max_value.append(row_max[name_column])
        
        row_min = df.agg({i: "min"}).collect()[0]
        name_column = 'min('+i+')'
        min_value.append(row_min[name_column])
    max_min.append(max_value)
    max_min.append(min_value)
    
#     luu gia tri khoi tao vi tri dau
    k_position =  random_init(max_min,k,len_columns)

#     them column group vào df với giá trị -1
    df=df.withColumn('group', lit(-1))
#     phan lop
    while True:
      # lặp từng phần tử, truyền tất cả row vào
        df=df.rdd.map(classtify).toDF()
        k_position = find_positions(df,k,columns)
# nếu sum = 0 là k thay đổi cụm
        if df.agg(F.sum("status")).collect()[0][0] == 0 :
            break
    return df.drop('status')


In [ ]:
# phuong phap eblow method

def distance_square(position,k_position):
    len_columns=len(k_position[0])
    # index là cụm 0 or cụm 1, position là 1 điểm, len_columns là cột group
    index = position[len_columns]
    distance = 0
    for j in range (0,len_columns):
        distance = distance + (position[j] - k_position[index][j])**2
    return distance
def Eblow_method(df,k):
    from pyspark.sql import Row
    
    def add_distance_square(row):
        rd=row.asDict()
        # số cột của df = những cột thuộc + cột group
        len_columns=len(k_position[0]) +1
        # position là 1 row của df
        position=[]
        # lặp theo từng cột
        for i in range (0,len_columns):
            position.append(row[i])
        distance = distance_square(position,k_position)
        # thêm cột distance_square vào df
        rd["distance_square"]=distance
        new_row=Row(**rd)  
        return new_row
    
    columns = df.columns
    columns.pop()
    # tìm lại trung tâm cụm
    k_position= find_positions(df,k,columns)
 
    df=df.rdd.map(add_distance_square).toDF()
    df.drop('distance_square')
    
    import pyspark.sql.functions as F     
    
    # tính tổng các khoảng cách
    score =   df.agg(F.sum("distance_square")).collect()[0][0]

    return score


In [ ]:
df1=df
start_time = time.time()
df1 = k_mean(df1,2)
# main()
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
Eblow_method(df1,2)

In [ ]:
# df1.collect()

In [ ]:
df1[df1['group']==0].show()

In [ ]:
pandas_df = df1.toPandas()


In [ ]:
sns.scatterplot(data=pandas_df, x="avg_wind_direction", y="relative_humidity", hue="group",palette = 'Set2')

Tất cả các thuộc tính đang được xem xét là số hoặc số rời rạc, do đó chúng ta cần chuyển đổi chúng thành các đặc trưng bằng cách sử dụng Vector Assembler. Vector Assembler là  transformer chuyển đổi một tập hợp các features thành một cột vectơ duy nhất thường được gọi là một mảng các features. features ở đây là các cột. Vì id khách hàng là số nhận dạng sẽ không được sử dụng để phân cụm, trước tiên chúng tôi trích xuất các cột bắt buộc bằng cách sử dụng .columns, chuyển nó làm đầu vào cho Vector Assembler và sau đó sử dụng transform() để chuyển đổi các cột đầu vào thành một vectơ duy nhất cột được gọi là một feature.

In [ ]:
from pyspark.ml.feature import VectorAssembler
df2=df
df.columns


In [ ]:
assemble=VectorAssembler(inputCols=[
 'air_temp',
 'avg_wind_direction',
 'max_wind_direction',
 'min_wind_direction',
 'relative_humidity',
  ], outputCol='features')
assembled_data=assemble.transform(df2)
assembled_data.show(2)

Bây giờ tất cả các cột được chuyển đổi thành một vectơ đặc trưng duy nhất, chúng tôi cần chuẩn hóa dữ liệu để đưa chúng về quy mô có thể so sánh được. Ví dụ. Cân bằng có thể có thang đo từ 10–1000 trong khi tần số cân bằng có thang đo từ 0–1. Khoảng cách Euclide luôn bị tác động nhiều hơn bởi các biến ở thang đo cao hơn, do đó, điều quan trọng là phải mở rộng các biến ra.

In [ ]:
from pyspark.ml.feature import StandardScaler
scale=StandardScaler(inputCol='features',outputCol='standardized')
data_scale=scale.fit(assembled_data)
data_scale_output=data_scale.transform(assembled_data)
data_scale_output.show(2)

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
silhouette_score=[]
evaluator = ClusteringEvaluator(predictionCol='prediction', featuresCol='standardized', \
                                metricName='silhouette', distanceMeasure='squaredEuclidean')
for i in range(2,10):
    
    KMeans_algo=KMeans(featuresCol='standardized', k=i)
    
    KMeans_fit=KMeans_algo.fit(data_scale_output)
    
    output=KMeans_fit.transform(data_scale_output)
    
    
    
    score=evaluator.evaluate(output)
    
    silhouette_score.append(score)
    
    print("Silhouette Score:",score)

In [ ]:
#Visualizing the silhouette scores in a plot
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1,1, figsize =(8,6))
ax.plot(range(2,10),silhouette_score)
ax.set_xlabel("k")
ax.set_ylabel("cost")

In [ ]:
data_scale_output.show(2)

In [ ]:
start_time = time.time()
# main()

KMeans_algo1=KMeans(featuresCol='standardized', k=5 )
    
KMeans_fit1=KMeans_algo1.fit(data_scale_output)
    
output1=KMeans_fit1.transform(data_scale_output)

print("--- %s seconds ---" % (time.time() - start_time))



In [ ]:
print("Silhouette Score:",evaluator.evaluate(output1))


In [ ]:
KMeans_fit1

In [ ]:
output1.show(5)

In [ ]:
sns.scatterplot(data=output1.toPandas()  , x="avg_wind_direction", y="relative_humidity", hue="prediction",palette = 'Set2')